In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

Tue Apr 12 10:20:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os 

os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/MyDrive/kaggle_dataset'

In [4]:
%cd /content/gdrive/MyDrive/kaggle_dataset

/content/gdrive/MyDrive/kaggle_dataset


In [5]:
!ls

kaggle.json  land-use


In [6]:
!mkdir RNN

In [10]:
%cd RNN
!kaggle competitions download -c fake-news

/content/gdrive/MyDrive/kaggle_dataset/RNN
 71% 33.0M/46.5M [00:00<00:00, 105MB/s] 
100% 46.5M/46.5M [00:00<00:00, 107MB/s]


In [12]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [14]:
import pandas as pd 
import numpy as np 

In [15]:
data = pd.read_csv("train.csv")

In [17]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
data = data.dropna()

In [19]:
X = data.drop(columns='label')
y = data['label']

In [20]:
import tensorflow as tf 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [21]:
messages = X.copy()
messages['title'][10]

'Obama’s Organizing for Action Partners with Soros-Linked ‘Indivisible’ to Disrupt Trump’s Agenda'

In [22]:
messages.reset_index(inplace=True)

In [23]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
vocab_size = 5000

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

    print(f"\rProgress: {i+1}",end='')

Progress: 18285

In [27]:
onehot_enc=[one_hot(words,vocab_size)for words in corpus] 

In [29]:
sentence_length=20
embedded_docs=pad_sequences(onehot_enc,padding='pre',maxlen=sentence_length)
print(embedded_docs)

[[   0    0    0 ... 1648  423 3563]
 [   0    0    0 ... 1259  510 2248]
 [   0    0    0 ...  885 2884 3868]
 ...
 [   0    0    0 ...  652 4778 2609]
 [   0    0    0 ... 3124  708 4821]
 [   0    0    0 ... 4459 3631  841]]


In [30]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
X_full = np.array(embedded_docs)
y_full = np.array(y)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.20, random_state=42)

In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 11s 10ms/step - loss: 0.3120 - accuracy: 0.8539 - val_loss: 0.2156 - val_accuracy: 0.9106
Epoch 2/10
229/229 [==============================] - 2s 8ms/step - loss: 0.1517 - accuracy: 0.9416 - val_loss: 0.2143 - val_accuracy: 0.9027
Epoch 3/10
229/229 [==============================] - 2s 8ms/step - loss: 0.1076 - accuracy: 0.9606 - val_loss: 0.2321 - val_accuracy: 0.9155
Epoch 4/10
229/229 [==============================] - 2s 7ms/step - loss: 0.0764 - accuracy: 0.9714 - val_loss: 0.2795 - val_accuracy: 0.9109
Epoch 5/10
229/229 [==============================] - 2s 7ms/step - loss: 0.0549 - accuracy: 0.9810 - val_loss: 0.3145 - val_accuracy: 0.9054
Epoch 6/10
229/229 [==============================] - 2s 7ms/step - loss: 0.0359 - accuracy: 0.9887 - val_loss: 0.3504 - val_accuracy: 0.9068
Epoch 7/10
229/229 [==============================] - 2s 7ms/step - loss: 0.0240 - accuracy: 0.9923 - val_loss: 0.4243 - val_accuracy: 0.9048
Epoc

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
y_pred = model.predict(X_test)
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91      2082
           1       0.86      0.93      0.89      1575

    accuracy                           0.90      3657
   macro avg       0.90      0.91      0.90      3657
weighted avg       0.91      0.90      0.90      3657

